In [4]:
!pip install pymongo pandas

  Using cached pymongo-4.15.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (22 kB)
  Using cached dnspython-2.8.0-py3-none-any.whl.metadata (5.7 kB)
Using cached pymongo-4.15.3-cp313-cp313-macosx_11_0_arm64.whl (975 kB)
Using cached dnspython-2.8.0-py3-none-any.whl (331 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pymongo]m1/2 [pymongo]


In [5]:
from pymongo import MongoClient
import pandas as pd
import json

In [6]:
#connect to MongoDB server
client = MongoClient("mongodb://localhost:27017/")

#switch to the 'nam' database
db = client["nam"]

#list databases to confirm connection
print(client.list_database_names())


['admin', 'config', 'local']


In [19]:
#load JSON file
with open("companies.json") as c:
    data = json.load(c)


In [20]:
#view one record to confirm keys/structure
print(data[0])


{'company': 'Apple', 'city': 'Cupertino', 'state': 'California', 'country': 'USA', 'description': 'American multinational technology company that designs, develops, and sells consumer electronics, computer software, and online services', 'employees': 164000, 'founded': 1976, 'revenue': 394.33}


In [ ]:
#create a collection in MongoDB named "companies"
companies_col = db["companies"]


In [22]:
#insert all records into the collection
companies_col.insert_many(data)


InsertManyResult([ObjectId('68fbd3cf2fe35c7c13b9ca0b'), ObjectId('68fbd3cf2fe35c7c13b9ca0c'), ObjectId('68fbd3cf2fe35c7c13b9ca0d'), ObjectId('68fbd3cf2fe35c7c13b9ca0e'), ObjectId('68fbd3cf2fe35c7c13b9ca0f'), ObjectId('68fbd3cf2fe35c7c13b9ca10'), ObjectId('68fbd3cf2fe35c7c13b9ca11'), ObjectId('68fbd3cf2fe35c7c13b9ca12'), ObjectId('68fbd3cf2fe35c7c13b9ca13'), ObjectId('68fbd3cf2fe35c7c13b9ca14'), ObjectId('68fbd3cf2fe35c7c13b9ca15'), ObjectId('68fbd3cf2fe35c7c13b9ca16'), ObjectId('68fbd3cf2fe35c7c13b9ca17'), ObjectId('68fbd3cf2fe35c7c13b9ca18'), ObjectId('68fbd3cf2fe35c7c13b9ca19')], acknowledged=True)

In [23]:
#verify the insertion 
#count documents
print("Total documents:", companies_col.count_documents({}))

#view first document
print(companies_col.find_one())


Total documents: 15
{'_id': ObjectId('68fbd3cf2fe35c7c13b9ca0b'), 'company': 'Apple', 'city': 'Cupertino', 'state': 'California', 'country': 'USA', 'description': 'American multinational technology company that designs, develops, and sells consumer electronics, computer software, and online services', 'employees': 164000, 'founded': 1976, 'revenue': 394.33}


In [27]:
#find all companies founded in or after 1990 and sort from fewest to most employees
#query for greater or equal to 1990
query = {"founded": {"$gte": 1990}}
#inclusion & exclusion flags for variables (0 & 1)
projection = {"_id": 0, "company": 1, "founded": 1, "employees": 1}

results = companies_col.find(query, projection).sort("employees", 1)

for r in results:
    print(r)

{'company': 'Tesla', 'employees': 127855, 'founded': 2003}
{'company': 'Alphabet', 'employees': 190234, 'founded': 1998}
{'company': 'Alibaba Group', 'employees': 239740, 'founded': 1999}
{'company': 'Amazon', 'employees': 1541000, 'founded': 1994}


In [ ]:
#find the totals of annual revenues of the companies headquartered in the US and Europe (Germany, UK, Switzerland)
#$match to filter by condition
#$in to match any of the countries listed
#$group to aggregate the sum of revenue

pipeline_us = [
    {"$match": {"country": "USA"}},
    {"$group": {"_id": "USA", "total_revenue_billion": {"$sum": "$revenue"}}}
]

result_us = list(companies_col.aggregate(pipeline_us))
print(result_us)

pipeline_europe = [
    {"$match": {"country": {"$in": ["Germany", "United Kingdom", "Switzerland"]}}},
    {"$group": {"_id": "Europe", "total_revenue_billion": {"$sum": "$revenue"}}}
]

result_europe = list(companies_col.aggregate(pipeline_europe))
print(result_europe)


[{'_id': 'USA', 'total_revenue_billion': 1470.8799999999999}]
[{'_id': 'Europe', 'total_revenue_billion': 571.37}]


In [ ]:
#inspect the description variable to confirm what to match for question 3
for doc in companies_col.find(
    {"description": {"$regex": "crude oil reserves", "$options": "i"}},
    {"_id": 0, "company": 1, "country": 1, "description": 1}
):
    print(doc, "\n")


{'company': 'Saudi Aramco', 'country': 'Saudi Arabia', 'description': "Saudi Arabian public petroleum and natural gas company with the world's second-largest proven crude oil reserves."} 



In [31]:
#find the country where the company that has the world's second largest proven crude oil reserves headquartered
query = {
    "description": {"$regex": "second.*crude oil reserves", "$options": "i"}
}

projection = {"_id": 0, "company": 1, "country": 1}

result = companies_col.find_one(query, projection)


print("Country:", result["country"])


Country: Saudi Arabia


In [33]:
#find the company established before 1970, which has the largest annual revenue
#filter for founded < 1970
results = companies_col.find(
    {"founded": {"$lt": 1970}},
    {"_id": 0, "company": 1, "founded": 1, "revenue": 1}
).sort("revenue", -1).limit(1)

for r in results:
    print(r)

{'company': 'Saudi Aramco', 'founded': 1933, 'revenue': 552.25}


In [34]:
#find the the total revenue and the total number of employees of all energy (oil and/or gas) companies.
pipeline_energy = [
    {"$match": {"description": {"$regex": "oil|gas", "$options": "i"}}},
    {"$group": {
        "_id": "Energy sector",
        "total_revenue_billion": {"$sum": "$revenue"},
        "total_employees": {"$sum": "$employees"}
    }}
]

result_energy = list(companies_col.aggregate(pipeline_energy))
print(result_energy)


[{'_id': 'Energy sector', 'total_revenue_billion': 1212.84, 'total_employees': 1581100}]
